**лабораторная работа 1.**

Выполнила: Клитная Анастасия группа м103

In [52]:
import numpy as np
import pandas as pd
import re  #для удобства работы со строками

теперь загрузим сюда файл

перед работой не забыть присоеденить файл\скачать его

In [45]:
name = input('введите название файла ') #если есть возможность ввести с клавиатуры
#name = 'mtesrl_20150626_MD0000600002_stats.txt' #файл из папки с заданием

введите название файла test1.txt


In [46]:
#откроем наш файл и прочитаем данные с него
with open(name, 'r') as file:
        lines = file.readlines()
# Отфильтруем строки, содержащие нужные данные(выкенет лишнее)
data = []
for line in lines:
    if re.match(r'^\[\d{2}:\d{2}:\d{2}\]', line):  # строки с данными начинаются с времени(видно из примера структуры файла)
        parts = line.split()  # разделим строку на части, где разделители пробелы
        event = parts[1]  # выделим EVENT
        avgtsmr = int(parts[-1])  # "AVGTSMR" - последний непустой столбик
        data.append((event, avgtsmr))


In [47]:
# Создаем DataFrame или если проще саму таблицу в пандас
df = pd.DataFrame(data, columns=['EVENT', 'AVGTSMR'])


In [48]:
#просто посмотрим структуру таблицы
df

,EVENT,AVGTSMR
0,AAAAA,476
1,ORDER,313
2,ORDER,217
3,ORDER,222
4,ORDER,206
...,...,...
999995,ORDER,181
999996,ORDER,177
999997,ORDER,225
999998,ORDER,182


In [49]:
# Группируем данные по типу транзакции
stats = df.groupby('EVENT')['AVGTSMR'].agg(['min', 'median','max', lambda x: np.percentile(x, 90),
                                               lambda x: np.percentile(x, 99),
                                               lambda x: np.percentile(x, 99.9)],)

# Переименуем колонки
stats.columns = ['min', '50%','max', '90%', '99%', '99.9%']
T = {}
# Форматируем вывод
for event in stats.index:
    print(f"{event} min={int(stats.loc[event, 'min'])} 50%={int(stats.loc[event, '50%'])} "
          f"90%={int(stats.loc[event, '90%'])} 99%={int(stats.loc[event, '99%'])} "
          f"99.9%={int(stats.loc[event, '99.9%'])}")
    time_min = int(stats.loc[event, 'min'])
    time_max = int(stats.loc[event, 'max'])#99.9%'])
    T[event] = [time_min, time_max]


AAAAA min=203 50%=205 90%=369 99%=465 99.9%=474
ORDER min=169 50%=190 90%=249 99%=271 99.9%=474


In [50]:
#просто посмотрим на крайние значения всей таблицы
print(T)

{'AAAAA': [203, 476], 'ORDER': [169, 7835]}


In [51]:
table = df.groupby('EVENT')#['AVGTSMR']
result = []
#необходим минимум кратный 5
for even, gr in table:
  while T[even][0] % 5 != 0:
    T[even][0]+=1

for even, gr in table:
  size_gr = len(gr)
  res = []
  for i in range(T[even][0],T[even][1], 5):
    tmp = gr['AVGTSMR']
    #необходимо отфильтровать данные по времени для нашей таблицы
    transno = len(gr[tmp == i]) # Количество транзакций с конкретным временем
    weight = transno / size_gr * 100 if size_gr > 0 else 0 #Процент от общего количества транзакций
    percent = gr[tmp <= i].shape[0] / size_gr * 100 if size_gr > 0 else 0
    res.append((i, transno, weight, percent))

  res_df = pd.DataFrame(res, columns = ['ExecTime', 'TransNo', 'Weight,%', 'Percent'])
  res_df['EVENT'] = even
  result.append(res_df)

final_result = pd.concat(result)
#для красоты
final_result = final_result[['EVENT', 'ExecTime', 'TransNo', 'Weight,%', 'Percent']]


In [39]:
print(T)

{'AAAAA': [205, 476], 'ORDER': [170, 7835]}


In [40]:
tab = final_result.groupby('EVENT')
for even, group in tab:
  print(f"Таблица для EVENT: {event}")
  print(group)

Таблица для EVENT: ORDER
    EVENT  ExecTime  TransNo  Weight,%  Percent
0   AAAAA       205        1      20.0     60.0
1   AAAAA       210        1      20.0     80.0
2   AAAAA       215        0       0.0     80.0
3   AAAAA       220        0       0.0     80.0
4   AAAAA       225        0       0.0     80.0
5   AAAAA       230        0       0.0     80.0
6   AAAAA       235        0       0.0     80.0
7   AAAAA       240        0       0.0     80.0
8   AAAAA       245        0       0.0     80.0
9   AAAAA       250        0       0.0     80.0
10  AAAAA       255        0       0.0     80.0
11  AAAAA       260        0       0.0     80.0
12  AAAAA       265        0       0.0     80.0
13  AAAAA       270        0       0.0     80.0
14  AAAAA       275        0       0.0     80.0
15  AAAAA       280        0       0.0     80.0
16  AAAAA       285        0       0.0     80.0
17  AAAAA       290        0       0.0     80.0
18  AAAAA       295        0       0.0     80.0
19  AAAAA      